In [ ]:
# -*- coding: utf-8 -*-
# 目的：匯出 LGBM（含校準若支援，否則基礎）與 TCN（Keras→ONNX）為 ONNX，並用 ORT 驗證

from pathlib import Path
import numpy as np
import pandas as pd
import joblib
import onnx
import onnxruntime as ort

# LGBM 轉換工具
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# TCN 轉換工具
import tensorflow as tf
from tensorflow import keras
import tf2onnx
from tcn import TCN

# ========== 0) 路徑 ==========
FEAT_PATH = Path("./data/feat_6h.parquet")
LABEL_PATH = Path("./data/label_6h.parquet")

ART_TCN = Path("./artifacts_tcn")
ART_LGBM = Path("./artifacts_lgbm")
ONNX_DIR = Path("./onnx_export")
ONNX_DIR.mkdir(parents=True, exist_ok=True)

LOOKBACK = 16

# ========== 1) 載入資料與特徵維度 ==========
feat = pd.read_parquet(FEAT_PATH)
label = pd.read_parquet(LABEL_PATH)
df = feat.merge(label, on="time", how="inner").sort_values("time").reset_index(drop=True)

X_all = df.drop(columns=["time", "y_dir_6h", "y_tail_6h"]).values.astype(np.float32)
feature_dim = X_all.shape[1]

# ========== 2) 匯出 LightGBM（嘗試含校準，不支援則退回基礎模型） ==========
lgbm_model = joblib.load(ART_LGBM / "lgbm_model.pkl")
lgbm_cal = joblib.load(ART_LGBM / "lgbm_calibrator.pkl")
lgbm_scaler = joblib.load(ART_LGBM / "scaler.pkl")

# 先將縮放融合進 sklearn Pipeline（校準器是否支援依 skl2onnx 版本而定）
from sklearn.pipeline import Pipeline
pipe_base = Pipeline(steps=[("scaler", lgbm_scaler), ("lgbm", lgbm_model)])

initial_type = [("input", FloatTensorType([None, feature_dim]))]

try:
    onnx_lgbm = convert_sklearn(pipe_base, initial_types=initial_type, target_opset=13)
    (ONNX_DIR / "lgbm_model.onnx").write_bytes(onnx_lgbm.SerializeToString())
    lgbm_onnx_path = ONNX_DIR / "lgbm_model.onnx"
    print("[OK] LGBM base ONNX 匯出完成")
except Exception as e:
    # 若轉換失敗，退回只轉換 LGBM 模型（不含 scaler）
    print(f"[WARN] Pipeline 轉換失敗，改轉 LGBM-only: {e}")
    onnx_lgbm = convert_sklearn(lgbm_model, initial_types=[("input", FloatTensorType([None, feature_dim]))], target_opset=13)
    (ONNX_DIR / "lgbm_model.onnx").write_bytes(onnx_lgbm.SerializeToString())
    lgbm_onnx_path = ONNX_DIR / "lgbm_model.onnx"

# 校準器（Isotonic）通常不被 skl2onnx 支援，部署時請於上層進行後處理：
# calibrated_prob = iso.transform(prob_from_onnx)

# ========== 3) 匯出 TCN（Keras → ONNX） ==========
tcn_model = keras.models.load_model(ART_TCN / "tcn_model.h5", custom_objects={"TCN": TCN})

spec = (tf.TensorSpec([None, LOOKBACK, feature_dim], tf.float32, name="input"),)
onnx_model, _ = tf2onnx.convert.from_keras(
    tcn_model,
    input_signature=spec,
    opset=13,
    output_path=str(ONNX_DIR / "tcn_model_raw.onnx")
)
tcn_onnx_path = ONNX_DIR / "tcn_model_raw.onnx"
print("[OK] TCN raw ONNX 匯出完成")

# ========== 4) ONNX 驗證與對比 ==========
# LGBM ONNX 對比
sess_lgbm = ort.InferenceSession(str(lgbm_onnx_path), providers=["CPUExecutionProvider"])
# 取樣本特徵
X_sample = X_all[:8]
# pipeline 內部若含 scaler，直接比較；否則需先縮放再比較
if any(isinstance(step, StandardError) for name, step in getattr(pipe_base, "steps", [])):  # 占位自檢，實務上直接試推理
    pass
inp = {"input": X_sample.astype(np.float32)}
onnx_out = sess_lgbm.run(None, inp)[0].ravel()

# 原生 LGBM 輸出（配合 scaler）
proba_native = lgbm_model.predict_proba(lgbm_scaler.transform(X_sample))[:, 1]
print("LGBM native vs ONNX (first 5):")
print(np.vstack([proba_native[:5], onnx_out[:5]]).T)

# TCN ONNX 對比
from sklearn.preprocessing import StandardScaler  # 僅為型別引用
tcn_scaler = joblib.load(ART_TCN / "scaler.pkl")

def make_sequences(X2d: np.ndarray, lookback: int) -> np.ndarray:
    seq = []
    for i in range(lookback, len(X2d)):
        seq.append(X2d[i - lookback:i, :])
    return np.asarray(seq, dtype=np.float32)

X_scaled = tcn_scaler.transform(X_all)
X_seq = make_sequences(X_scaled, LOOKBACK)
X_seq_sample = X_seq[:8]

sess_tcn = ort.InferenceSession(str(tcn_onnx_path), providers=["CPUExecutionProvider"])
onnx_out_tcn = sess_tcn.run(None, {"input": X_seq_sample.astype(np.float32)})[0].ravel()
keras_out_tcn = tcn_model.predict(X_seq_sample, verbose=0).ravel()

print("TCN native vs ONNX (first 5):")
print(np.vstack([keras_out_tcn[:5], onnx_out_tcn[:5]]).T)

print({
    "onnx_lgbm_path": str(lgbm_onnx_path),
    "onnx_tcn_path": str(tcn_onnx_path),
    "note": "若需校準，請於推理端套用 isotonic_calibrator.transform(prob)"
})
